# PyTorch 基础 : 神经网络包nn和优化器optim
torch.nn是专门为神经网络设计的模块化接口。nn构建于Autograd之上，可用来定义和运行神经网络。
这里我们主要介绍几个一些常用的类。

**约定：torch.nn 我们为了方便使用，会为他设置别名为nn，本章除nn以外还有其他的命名约定**

In [2]:
import torch
import torch.nn as nn
print(torch.__version__)

1.10.0+cu111


除了nn别名以外，我们还引用了`nn.functional`，这个包中包含了神经网络中使用的一些常用函数，这些函数的特点是，`没有可学习的参数(如ReLU，pool，DropOut等)`，这些函数可以放在构造函数中，也可以不放，但是这里建议不放。

一般情况下我们会**将nn.functional 设置为大写的F**，这样缩写方便调用

如果模型有`可学习的参数`时，最好使用`nn.Module`；

否则既可以使用nn.functional也可以使用nn.Module，二者在性能上没有太大差异，具体的使用方式取决于个人喜好。

由于`激活函数（ReLu、sigmoid、Tanh）`、`池化（MaxPool）`等层`没有可学习的参数`，可以使用对应的`functional`函数，而`卷积`、`全连接`等`有可学习参数`的网络建议使用`nn.Module`。

虽然`dropout没有可学习参数`，但`建议还是使用nn.Dropout`而不是nn.functional.dropout，因为`dropout在训练和测试两个阶段的行为有所差别`，使用nn.Module对象能够通过model.eval操作加以区分。


In [3]:
import torch.nn.functional as F

## 定义一个网络
PyTorch中已经为我们准备好了现成的网络模型，只要`继承nn.Module`，并`实现它的forward方法`，PyTorch会根据autograd，`自动实现backward函数`，在`forward函数`中可使用`任何tensor支持的函数`，还可以使用`if、for循环、print、log等Python语法`，写法和标准的Python写法一致。

In [5]:
class Net(nn.Module): ## 继承nn.Module类，这样就可以使用其中设置好的网络了
  def __init__(self):
  ## nn.Module子类的函数必须在构造函数中执行父类的构造函数
    super(Net,self).__init__()
  ## 这里这个supre是因为Net作为nn.Module的子类，需要继承父类的初始化方法
    # 卷积层 '1'表示输入图片为单通道， '6'表示输出通道数(6个卷积核)，'3'表示卷积核为3*3
    self.conv1 = nn.Conv2d(1, 6, 3) 
    #线性层，输入1350个特征，输出10个特征
    self.fc1  = nn.Linear(1350, 10)  #这里的1350是如何计算的呢？这就要看后面的forward函数
  
  ## forward propogation
  def forward(self, x): 
    print(x.size()) ## 打印input的size,结果：[1, 1, 32, 32]
    # 卷积 -> 激活 -> 池化 
    x=self.conv1(x) ## 使得x通过第一个定义好的卷积层1
    x=F.relu(x) ## 卷积完后一般过一个relu
    print(x.size()) ## 结果：[1, 6, 30, 30]

    x=F.max_pool2d(x,(2,2)) ## 池化，计算结果是15
    x = F.relu(x)
    print(x.size()) # 结果：[1, 6, 15, 15]

    ## reshape，‘-1’表示自适应
    ## 这里做的就是压扁的操作 就是把后面的[1, 6, 15, 15]压扁，变为 [1, 1350]
    x = x.view(x.size()[0], -1) ## 变成一个列向量
    print(x.size()) # 这里就是fc1层的的输入1350 
    x = self.fc1(x)        
    return x

net=Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1350, out_features=10, bias=True)
)


网络的可学习参数通过`net.parameters()`返回

In [6]:
for parameters in net.parameters():
  print(parameters)

Parameter containing:
tensor([[[[-0.3132, -0.2879, -0.2701],
          [ 0.1968, -0.1522,  0.0222],
          [-0.3314, -0.2016,  0.2558]]],


        [[[ 0.0338, -0.2085, -0.1017],
          [-0.0640,  0.1522,  0.3060],
          [-0.2275, -0.1613, -0.1478]]],


        [[[-0.1396,  0.0140, -0.2937],
          [-0.1972,  0.1084,  0.3069],
          [ 0.1732, -0.1109,  0.2003]]],


        [[[-0.0607, -0.1636, -0.0384],
          [ 0.0381,  0.0806,  0.1708],
          [-0.1295,  0.0415, -0.2344]]],


        [[[ 0.2609, -0.2931,  0.1968],
          [ 0.0670,  0.3028,  0.2024],
          [ 0.2918,  0.2290, -0.2034]]],


        [[[-0.2479, -0.1746, -0.1352],
          [-0.3297,  0.2654, -0.0288],
          [-0.2869, -0.2976,  0.2436]]]], requires_grad=True)
Parameter containing:
tensor([ 0.2910, -0.0292, -0.2471, -0.1848,  0.1137,  0.0102],
       requires_grad=True)
Parameter containing:
tensor([[-0.0053,  0.0017,  0.0231,  ..., -0.0134, -0.0143,  0.0183],
        [ 0.0126, -0.0024,  0

`net.named_parameters`可同时返回`可学习的参数`及`名称`。

In [7]:
for name,parameters in net.named_parameters():
    print(name,':',parameters.size())

conv1.weight : torch.Size([6, 1, 3, 3])
conv1.bias : torch.Size([6])
fc1.weight : torch.Size([10, 1350])
fc1.bias : torch.Size([10])


forward函数的输入和输出都是Tensor

In [9]:
input = torch.randn(1, 1, 32, 32) # 这里的对应前面forward的输入是32
out = net(input)
out.size()


torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])


torch.Size([1, 10])

In [10]:
input.size()

torch.Size([1, 1, 32, 32])

在反向传播前，先要将所有`参数的梯度清零`

In [11]:
net.zero_grad() 
out.backward(torch.ones(1,10)) # 反向传播的实现是PyTorch自动实现的，我们只要调用这个函数即可

**注意**:torch.nn只支持mini-batches，不支持一次只输入一个样本，即`一次必须是一个batch`。

也就是说，就算我们输入一个样本，也会对样本进行分批，所以，所有的输入都会增加一个维度，我们对比下刚才的input，nn中定义为3维，但是我们人工创建时多`增加了一个维度`，变为了4维，最`前面的1即为batch-size`

## 损失函数
在`nn`中PyTorch还预制了常用的`损失函数`，下面我们用`MSELoss`用来计算均方误差

In [13]:
y = torch.arange(0,10).view(1,10).float() ## 虚拟定义一个y，作为监督学习样本的label
loss_fn = nn.MSELoss()
loss = loss_fn(out, y)
#loss是个scalar，我们可以直接用item获取到他的python类型的数值
print(loss.item()) 

25.067508697509766


并且这里loss是一个标量，所以可以直接`loss.backward()`

## 优化器
在反向传播计算完所有参数的梯度后，还需要使用优化方法来更新网络的权重和参数，例如随机梯度下降法(SGD)的更新策略如下：

`weight = weight - learning_rate * gradient`

在`torch.optim`中实现大多数的优化方法，例如`RMSProp`、`Adam`、`SGD`等，下面我们使用SGD做个简单的样例

In [14]:
import torch.optim

In [15]:
out = net(input) # 这里调用的时候会打印出我们在forword函数中打印的x的大小
loss_fn = nn.MSELoss()
loss = loss_fn(out, y)
#新建一个优化器，SGD只需要要调整的参数和学习率
optimizer = torch.optim.SGD(net.parameters(), lr = 0.01,momentum=0.9) ## 带动量SGD
# 先梯度清零(与net.zero_grad()效果一样)
optimizer.zero_grad() 
loss.backward()

#更新参数
optimizer.step()

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])


这样，神经网络的数据的一个完整的传播就已经通过PyTorch实现了，下面一章将介绍PyTorch提供的数据加载和处理工具，使用这些工具可以方便的处理所需要的数据。

看完这节，大家可能对神经网络模型里面的一些参数的计算方式还有疑惑，这部分会在第二章 第四节 卷积神经网络有详细介绍，并且在第三章 第二节 MNIST数据集手写数字识别的实践代码中有详细的注释说明。